1. Load the paths

In [8]:
import os
import numpy as np

paths = [
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\amin_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin1\amin-1_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Ismayil\ismyil_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Marjan\marjan_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mina\mina_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mina 1\mina-1_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mina 3\mina-3_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mohammad\mohommad_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mona\mona_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Cole\cole_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Daniel\daniel_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Jack\jack_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\James\james_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Josh\josh_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Max\max_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Roddy\roddy_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Sam\sam_eeg.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\adam\adam_eeg.npy",
]

def head_along_first_axis(arr, n=5):
    if arr.ndim == 0:
        return arr  # scalar
    idx = (slice(0, min(n, arr.shape[0])),) + (slice(None),) * (arr.ndim - 1)
    return arr[idx]

for p in paths:
    print("\n" + "-"*88)
    print(p)
    try:
        # allow_pickle=True in case labels are object arrays (e.g., strings)
        arr = np.load(p, allow_pickle=True)
        print(f"Loaded: shape={arr.shape}, dtype={arr.dtype}")
        preview = head_along_first_axis(arr, n=5)
        print("First 5 lines (along axis 0):")
        print(preview)
    except FileNotFoundError:
        print("File not found.")
    except Exception as e:
        print(f"Error reading file: {e}")


----------------------------------------------------------------------------------------
C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\amin_eeg.npy
Loaded: shape=(265, 14, 200), dtype=object
First 5 lines (along axis 0):
[[['3682.051270' '3682.051270' '3681.538574' ... '3652.820557'
   '3691.794922' '3704.102539']
  ['4269.743652' '4265.641113' '4262.563965' ... '4226.153809'
   '4259.487305' '4272.820313']
  ['3976.410156' '3973.333252' '3971.281982' ... '3934.358887'
   '3971.794922' '3980.000000']
  ...
  ['4465.641113' '4471.794922' '4474.358887' ... '4427.692383'
   '4475.384766' '4477.436035']
  ['4141.025879' '4140.512695' '4141.538574' ... '4105.128418'
   '4157.436035' '4163.077148']
  ['4236.410156' '4233.846191' '4236.410156' ... '4216.410156'
   '4261.025879' '4264.102539']]

 [['3689.230713' '3682.564209' '3682.051270' ... '3665.128174'
   '3668.205078' '3662.051270']
  ['4261.025879' '4252.307617' '4249.230957' ... '4230.

2. Conacatenate arrays

In [10]:
expected = (265, 14, 200)

def load_eeg(path):
    """
    Load a .npy that may have been saved as an object array.
    Converts to a proper numeric ndarray with shape (265,14,200).
    """
    arr = np.load(path, allow_pickle=True)  # needed because source is object array
    if arr.dtype == object:
        # Try to stack element-wise if it's a 1D object array of (14,200) slices
        try:
            arr = np.stack([np.asarray(x) for x in arr], axis=0)
        except Exception:
            # Fall back to best-effort coercion
            arr = np.asarray(arr)
    if arr.shape != expected:
        raise ValueError(f"{os.path.basename(path)} has shape {arr.shape}, expected {expected}")
    # Ensure a consistent dtype (float32 is common for EEG; keep original if you want)
    if not np.issubdtype(arr.dtype, np.number):
        arr = arr.astype(np.float32, copy=False)
    return arr

# Load, validate, and concatenate
chunks = [load_eeg(p) for p in paths]
final_arr = np.concatenate(chunks, axis=0)


In [11]:
# Final sanity check: (4770, 14, 200)
assert final_arr.shape == (len(paths) * expected[0], expected[1], expected[2])
print("Final shape:", final_arr.shape, "dtype:", final_arr.dtype)

Final shape: (4770, 14, 200) dtype: float32


3. Save the final array

In [12]:
# Save alongside the first file
output_path = os.path.join(os.path.dirname(paths[0]), "final_eeg.npy")
np.save(output_path, final_arr)
print("Saved to:", output_path)

Saved to: C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\final_eeg.npy


In [13]:
import os
import shutil

# Use the output_path variable from earlier if it exists; otherwise fall back to the Amin folder
try:
    src = output_path  # from your previous run
except NameError:
    src = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\final_eeg.npy"

dst_dir = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data"
os.makedirs(dst_dir, exist_ok=True)
dst = os.path.join(dst_dir, "final_eeg.npy")

# Overwrite if file already exists
if os.path.exists(dst):
    os.remove(dst)

shutil.move(src, dst)  # use shutil.copy2(src, dst) if you prefer to keep the original
print(f"Final array moved to: {dst}")


Final array moved to: C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data\final_eeg.npy
